In [55]:
import pandas as pd
import os
import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import csr_matrix
import implicit
from sklearn.metrics.pairwise import pairwise_distances


In [56]:
os.getcwd()

'C:\\Users\\enabi\\OneDrive\\Desktop\\11182022\\DS_apply\\Projects_for_github\\Recommonder_system\\Recommender_system'

In [57]:
df_user_ratedmovies_timestamps= pd.read_csv('all/user_ratedmovies-timestamps.dat', sep='\t', encoding='ISO-8859-1')

In [58]:
df_user_ratedmovies_timestamps.head()

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000


In [59]:
df_user_ratedmovies_timestamps['movieID'].max()

65133

In [60]:
df_movies= pd.read_csv('all/movies.dat', sep='\t', encoding='ISO-8859-1')

In [61]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [62]:
df_movies['id'].max()

65133

In [63]:
#Obtain the ratings corresponding to some movies:

some_movies= ['The Matrix', 'Alien', 'The Godfather']
some_ids= {}
for mov in some_movies:
    some_ids[mov]=df_movies.loc[df_movies['title']==mov,'id'].values
some_ids

{'The Matrix': array([2571], dtype=int64),
 'Alien': array([1214], dtype=int64),
 'The Godfather': array([  858, 25934], dtype=int64)}

In [64]:
df_movies.loc[df_movies['id'] .isin ([2571, 1214, 858, 25934])]

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
787,858,The Godfather,68646,El padrino,http://ia.media-imdb.com/images/M/MV5BMTc3MDQz...,1972,godfather,9,66,66,...,100,8.4,11,11,0,100,4.4,172072,97,http://content9.flixster.com/movie/11/12/76/11...
1102,1214,Alien,78748,Alien - El octavo pasajero,http://ia.media-imdb.com/images/M/MV5BMTgzMDk3...,1979,american-alien,0,0,0,...,0,0,0,0,0,0,0,0,0,http://content7.flixster.com/movie/10/87/47/10...
2350,2571,The Matrix,133093,Matrix,http://ia.media-imdb.com/images/M/MV5BMjEzNjg1...,1999,matrix,7.4,125,108,...,86,6.2,28,19,9,67,3.6,207333,81,http://content7.flixster.com/movie/16/90/52/16...
8048,25934,The Godfather,68646,El padrino,http://ia.media-imdb.com/images/M/MV5BMTc3MDQz...,1972,godfather,9,66,66,...,100,8.4,11,11,0,100,4.4,172072,97,http://content9.flixster.com/movie/11/12/76/11...


In [65]:
#Obtain the ratings of 'Alien':
r_Alien=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==1214, 'rating'].mean()
r_Alien

3.9579395085066165

In [66]:
#Obtain the ratings of 'The Godfather':
r_Godfather=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([25934,858]), 'rating'].mean()
r_Godfather

4.334507042253521

In [67]:
#Obtain the ratings of 'The Matrix':
r_Matrix=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==2571, 'rating'].mean()
r_Matrix

4.173952095808383

In [68]:
# Obtain the users rated 'The Godfather'

user_Godfather =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([25934,858]), 'userID'].values

user_Godfather

array([   78,   175,   190, ..., 71509, 71525, 71534], dtype=int64)

In [69]:
# Obtain the users who rated 'Alien'

user_Alien =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==1214, 'userID'].values

user_Alien

array([   78,   175,   190, ..., 71483, 71497, 71525], dtype=int64)

In [70]:
# Obtain the users who rated 'The Matrix'

user_Matrix =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==2571, 'userID'].values

user_Matrix

array([   75,    78,   170, ..., 71525, 71529, 71534], dtype=int64)

In [71]:
# Obtain the users who rated the previous movies

distinct_users =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([2571, 1214, 858, 25934]), 'userID'].unique()

distinct_users


array([   75,    78,   170, ..., 71525, 71529, 71534], dtype=int64)

In [72]:
# Create a sparse matrix -- This matrix will include each item as a row, and the users will be the columns
# The value of each cell of the matrix will be filled with the user rating for each movie

df_mov= df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([2571, 1214, 858, 25934]), :]
df_mov

,userID,movieID,rating,timestamp
29,75,2571,4.5,1162161050000
98,78,858,4.0,1089332870000
149,78,1214,5.0,1083963994000
286,78,2571,5.0,1083964044000
612,170,2571,3.5,1162208300000
...,...,...,...,...
854959,71525,1214,5.0,1045440408000
855027,71525,2571,5.0,1001839814000
855396,71529,2571,2.5,1162098860000
855461,71534,858,5.0,1196645187000


In [73]:
matrix = df_mov.pivot(index='movieID', columns='userID', values='rating')

matrix

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5
25934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Since both 25934 and 858 are the movieId of the same movie: "The godfather"
# I want to merge them, first see what is in the movieId==25934, because it seems more sparse than the other one.

matrix.loc[25934].value_counts(dropna=False)

NaN    1820
3.5       1
4.5       1
Name: 25934, dtype: int64

In [85]:
# Now, we can transfer these two values to the movieId==858 (If they are not already full) and delete the movieId==25934 row.

filt=matrix.loc[25934].notna()
matrix.columns[filt]

Int64Index([51033, 52983], dtype='int64', name='userID')

In [86]:
matrix.loc[858, [51033, 52983]]

userID
51033    4.0
52983    5.0
Name: 858, dtype: float64

In [87]:
# I deleted the row movieId==25934

matrix= matrix.drop(25934, axis=0)

In [88]:
matrix

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [91]:
matrix_zero = matrix.copy()

In [92]:
matrix_zero

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [93]:
df_user_ratedmovies_timestamps['rating'].value_counts(dropna=False)

4.0    215773
3.0    155918
3.5    150582
4.5     88652
5.0     71680
2.5     62454
2.0     57188
1.0     21535
1.5     18328
0.5     13488
Name: rating, dtype: int64

In [94]:
matrix_zero.fillna(0, inplace=True)

In [95]:
matrix_zero

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,0.0,4.0,0.0,5.0,1.0,2.5,5.0,4.0,0.0,5.0,...,0.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0
1214,0.0,5.0,0.0,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,0.0,3.0,0.0,4.0,0.0,4.0,0.0,5.0,0.0,0.0
2571,4.5,5.0,3.5,5.0,5.0,4.0,0.0,4.0,4.5,4.5,...,5.0,5.0,0.0,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [96]:
# converting the matrix_zero which is a data frame in to a Matrix

Matt =matrix_zero.values

In [97]:
Matt

array([[0. , 4. , 0. , ..., 5. , 0. , 5. ],
       [0. , 5. , 0. , ..., 5. , 0. , 0. ],
       [4.5, 5. , 3.5, ..., 5. , 2.5, 4.5]])

In [98]:
# calculate the sparsity of the matrix

#first calculate the number of non zeros
#  np.count_nonzero(matrix_zero!=0)

#calculate the number of zeros
num_zeros = Matt.size - np.count_nonzero(Matt!=0)
sparsity = num_zeros / Matt.size

In [99]:
sparsity

0.2934504207830223

In [100]:
# It seems that it is NOT very sparse 

In [101]:
matrix.isna().sum(axis=1)

movieID
858     688
1214    764
2571    152
dtype: int64

In [104]:
# fillna for each movie whith thier mean

matrix2 = matrix.apply(lambda x: x.fillna(x.mean()), axis=1)
matrix2

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,4.335097,4.0,4.335097,5.0,1.0,2.5,5.000000,4.0,4.335097,5.0,...,4.335097,4.0,5.000000,5.0,5.00000,5.0,5.00000,5.0,4.335097,5.00000
1214,3.957940,5.0,3.957940,5.0,3.5,4.0,5.000000,4.0,4.500000,4.5,...,3.957940,3.0,3.957940,4.0,3.95794,4.0,3.95794,5.0,3.957940,3.95794
2571,4.500000,5.0,3.500000,5.0,5.0,4.0,4.173952,4.0,4.500000,4.5,...,5.000000,5.0,4.173952,4.0,5.00000,5.0,3.50000,5.0,2.500000,4.50000


In [105]:
# Compute a similarity score
# cosine_similarity using sklearn library

sk.metrics.pairwise.cosine_similarity(matrix2, Y=None, dense_output=True)

array([[1.        , 0.97952346, 0.97205951],
       [0.97952346, 1.        , 0.97219492],
       [0.97205951, 0.97219492, 1.        ]])

In [106]:
# cosine_similarity without using sklearn library

# convert matrix to a sparse matrix
#sparse_matrix = csr_matrix(matrix)

# calculate dot product between rows of the sparse matrix
dot_products = matrix2.dot(matrix2.T)

# calculate Euclidean norm of rows of the sparse matrix
row_norms = np.sqrt(np.sum(np.square(matrix2), axis=1))

# calculate cosine similarity
cosine_similarity = dot_products.multiply(1 / row_norms).T.multiply(1 / row_norms)

In [107]:
cosine_similarity

movieID,858,1214,2571
movieID,,,
858,1.000000,0.979523,0.972060
1214,0.979523,1.000000,0.972195
2571,0.972060,0.972195,1.000000


In [108]:
# convert ratings matrix to binary matrix (We shoul work with matxi that has the null values (I mean matrix not matrix2) because we need to convert it into a binary matrix 
#in which 1 indicates that a user has rated a movie and 0 indicates that a user has not rated the movie. 

binary_matrix = matrix.notnull().astype('int')

# calculate Jaccard similarity matrix
jaccard_sim_matrix = 1 - pairwise_distances(binary_matrix.values, metric='jaccard')

C:\Users\enabi\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [109]:
jaccard_sim_matrix

array([[1.        , 0.50136986, 0.56473214],
       [0.50136986, 1.        , 0.57142857],
       [0.56473214, 0.57142857, 1.        ]])

In [117]:
# Convert the ratings matrix to a sparse matrix
#sparse_ratings = csr_matrix(matrix.values)

# Compute the correlation matrix using pairwise_distances
correlation_matrix = pairwise_distances(matrix2, metric='correlation')

In [118]:
correlation_matrix

array([[0.        , 0.87698886, 0.96299151],
       [0.87698886, 0.        , 0.85005872],
       [0.96299151, 0.85005872, 0.        ]])